In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
from scipy.signal import sawtooth

#### The DAC sampling rate is set at 9.34 GSps. 
#### With the arb_rfsoc overlay, a total of $2^{20}$ points can be loaded into the memory of the FPGA. Though the points are sent as a 16 bit signed integer to the FPGA , the DAC converts only the upper 14 bits into analog output.

#### Within the FPGA the points are stored in effectively two RAMs, each of width 128 bits and a depth of $2^{16}$. The 128 bit width corresponds to 8 samples of 16 bit numbers. This implies that the two RAMs store $8*2*2^{16} = 2^{20}$ points. 
#### The read address sent to the RAM is increased by 1 at every clock cycle till it reaches MAX_POINTS (which is a value set by the user) where it is reset to zero and the increament at clock cycle continues. This way the RAM continuously sends the stored data to the DAC.
#### The current modules includes a control signal sent in through the PMOD pin 1. When the control signal is low, the RAM address counts from 0 to $2^{19}-1$, and when the control is high, the the address counter counts from $2^{19}$ to $2^{20}-1$. This way the output can be switched between two different waveforms (each containing $2^{19}$ sample points) with controlled timings.

In [ ]:
DAC_SR = 9.33888e9 #in GHz
DAC_amplitude = 2**15 #16 bit representation in 2's complement format
N_points = 2**19
MAX_POINTS = int(2**15)
t = 1/DAC_SR*np.arange(N_points)

In [ ]:
#Generating test signal
freq = 40.0e6 #in Hz
mysignal = 0.9*DAC_amplitude*np.sin(2*np.pi*freq*t)
mysignal = 0.8*DAC_amplitude*sawtooth(2*np.pi*freq*t)
mysignal = np.int16(mysignal)
mysignal

### Load the overlay into the FPGA and program the oscillators on the board to produce the appropriate clock signals. 
The LMK and LMX files were taken from https://github.com/Xilinx/RFSoC-PYNQ/tree/master/boards/RFSoC4x2/packages/tics/tics/register_txts

In [ ]:
from pynq import Overlay
import xrfclk
ol = Overlay("./package/awg_rfsoc_trig.bit") #Load the FPGA bit file 
xrfclk.set_ref_clks(lmk_freq = 245.76,lmx_freq = 491.52) #Programs the oscillators on the board to produce the appropriate clock signals

### AXI-GPIO IP blocks are used within the FPGA to send data to the FPGA and program the arbitrary waveform generator.
#### <u>we</u>: 2 bit signal. 1 -> write the data to RAM1, 2 -> write data to RAM2, 0 -> read data from both the RAMs
#### <u>MAX_POINTS</u> : 32 bit integer. $2^4$ * MAX_POINTS gives the number of points to output from the arbitrary waveform generator. It's value should be less than the number of points to be loaded onto the memory
#### <u>data_in</u> : 16 bit number
#### <u>row</u> : 15 bit number. Specifies the position on the RAMs to place the the 16-bit data_in.
#### <u>col</u> : 8 bit number. Specifies the position on the RAMs to place the the 16-bit data_in.

In [ ]:
from pynq.lib import AxiGPIO

we_instance = ol.ip_dict['we']
we_port = AxiGPIO(we_instance).channel1
we_port.setdirection("out")

max_instance = ol.ip_dict['MAX_POINTS']
max_port = AxiGPIO(max_instance).channel1
max_port.setdirection("out")

data_in_instance = ol.ip_dict['data_in']
data_in_port = AxiGPIO(data_in_instance).channel1
data_in_port.setdirection("out")

row_instance = ol.ip_dict['row']
row_port = AxiGPIO(row_instance).channel1
row_port.setdirection("out")

col_instance = ol.ip_dict['col']
col_port = AxiGPIO(col_instance).channel1
col_port.setdirection("out")

### This is the sequence in which data in the AXI-GPIOs are written to load the waveform onto the memory of the FPGA

In [ ]:
import time
start = time.time()

max_port.write(int(MAX_POINTS),0xffffffff)

count = 0
for row_val in range(32768):
    row_port[0:15].write(row_val)
    for we_val in [1,2]:
        we_port[0:2].write(we_val)
        for col_val in range(8):
            col_port[0:3].write(col_val)
            data_in_val = int(mysignal[count] & 0xffff)
            data_in_port[0:16].write(data_in_val)
            count += 1
data_in_port[0:16].write(0)
stop = time.time()
printf(f"Took {stop - start} s to write to RAM!")
we_port[0:2].write(0)